https://data.seattle.gov/Public-Safety/Terry-Stops/28ny-9ts8

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import time
import os, sys

%load_ext autoreload
%autoreload 2
pd.set_option('display.max_columns', 100)

In [2]:
path = os.path.join('..', 'data', 'raw', 'terry-stops.csv')
df = pd.read_csv(path, dtype='str')

# Strip spaces, as relevant
for col in df.columns:
    df[col] = df[col].str.strip()

In [3]:
df.iloc[:30]

,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,Subject Perceived Race,Subject Perceived Gender,Reported Date,Reported Time,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat
0,-,-1,20140000120677,92317,Arrest,None,7500,1984,M,Black or African American,Asian,Male,2015-10-16T00:00:00,11:32:00,-,-,-,SOUTH PCT 1ST W - ROBERT,N,N,South,O,O2
1,-,-1,20150000001463,28806,Field Contact,None,5670,1965,M,White,-,-,2015-03-19T00:00:00,07:59:00,-,-,-,NaN,N,N,-,-,-
2,-,-1,20150000001516,29599,Field Contact,None,4844,1961,M,White,White,Male,2015-03-21T00:00:00,19:12:00,-,-,-,NaN,N,-,-,-,-
3,-,-1,20150000001670,32260,Field Contact,None,7539,1963,M,White,-,-,2015-04-01T00:00:00,04:55:00,-,-,-,NaN,N,N,-,-,-
4,-,-1,20150000001739,33155,Field Contact,None,6973,1977,M,White,Black or African American,Male,2015-04-03T00:00:00,00:41:00,-,-,-,NaN,N,N,-,-,-
5,-,-1,20150000001755,33571,Field Contact,None,7402,1973,M,White,Black or African American,Male,2015-04-05T00:00:00,23:46:00,-,-,-,NaN,N,N,-,-,-
6,-,-1,20150000002351,45252,Field Contact,None,7591,1985,M,Hispanic or Latino,Other,Female,2015-05-20T00:00:00,21:39:00,-,-,-,WEST PCT 3RD W - MARY,N,N,-,-,-
7,-,-1,20150000002363,45182,Field Contact,None,7591,1985,M,Hispanic or Latino,White,Male,2015-05-20T00:00:00,22:40:00,-,-,-,WEST PCT 3RD W - MARY,N,N,-,-,-
8,-,-1,20150000002392,45365,Field Contact,None,7560,1986,M,White,White,Female,2015-05-22T00:00:00,07:39:00,-,-,-,SOUTH PCT 1ST W - R/S RELIEF,N,N,East,E,E2
9,-,-1,20150000002451,46430,Field Contact,None,7591,1985,M,Hispanic or Latino,-,-,2015-05-25T00:00:00,01:06:00,-,-,-,WEST PCT 3RD W - MARY,N,N,-,-,-


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47213 entries, 0 to 47212
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Subject Age Group         47213 non-null  object
 1   Subject ID                47213 non-null  object
 2   GO / SC Num               47213 non-null  object
 3   Terry Stop ID             47213 non-null  object
 4   Stop Resolution           47213 non-null  object
 5   Weapon Type               47213 non-null  object
 6   Officer ID                47213 non-null  object
 7   Officer YOB               47213 non-null  object
 8   Officer Gender            47213 non-null  object
 9   Officer Race              47213 non-null  object
 10  Subject Perceived Race    47213 non-null  object
 11  Subject Perceived Gender  47213 non-null  object
 12  Reported Date             47213 non-null  object
 13  Reported Time             47213 non-null  object
 14  Initial Call Type     

In [5]:
# ------ CLEAN DATA ONE COLUMN AT A TIME -------

# Subject Age Group
df['Subject Age Group'].replace('-', np.nan, inplace=True)

# Subject ID
df['Subject ID'].replace('-1', np.nan, inplace=True)

# Weapon Type
df['Weapon Type'].replace('-', 'None', inplace=True)
df['Weapon Type'].replace('Firearm Other', 'Firearm', inplace=True)
df['Weapon Type'].replace('Other Firearm', 'Firearm', inplace=True)
df['Weapon Type'].replace('Firearm (unk type)', 'Firearm', inplace=True)
df['Weapon Type'].replace('None/Not Applicable', 'None', inplace=True)
df['Weapon Type'].replace('Brass Knuckles', 'Club, Blackjack, Brass Knuckles', inplace=True)
df['Weapon Type'].replace('Blackjack', 'Club, Blackjack, Brass Knuckles', inplace=True)
df['Weapon Type'].replace('Club', 'Club, Blackjack, Brass Knuckles', inplace=True)
df['Weapon Type'].replace('Shotgun', 'Rifle', inplace=True)

# Officer Year of Birth
df['Officer YOB'] = df['Officer YOB'].astype('int')

# Officer Race
df['Officer Race'].replace('Unknown', 'Not Specified', inplace=True)

# Subject Perceived Race
df['Subject Perceived Race'].replace('-', 'Unknown', inplace=True)
df['Subject Perceived Race'].replace('Other', 'Unknown', inplace=True)

# Subject Perceived Gender
df['Subject Perceived Gender'].replace('Unable to Determine', np.nan, inplace=True)
df['Subject Perceived Gender'].replace('-', np.nan, inplace=True)
df['Subject Perceived Gender'].replace('Unknown', np.nan, inplace=True)
df['Subject Perceived Gender'].replace('Male', 'M', inplace=True)
df['Subject Perceived Gender'].replace('Female', 'F', inplace=True)
df['Subject Perceived Gender'].replace('Gender Diverse (gender non-conforming and/or transgender)', 'N', inplace=True)

# Reported Date
df['Reported Date'] = df['Reported Date'].astype('datetime64')

# Reported Time
if 'datetime.time' not in str(type(df['Reported Time'][0])): # Use if statement to avoid error when re-running
    df['Reported Time'] = [time.fromisoformat(t) for t in df['Reported Time']]
    
# Call Type
df['Call Type'].replace('TEXT MESSAGE', '-', inplace=True)
df['Call Type'].replace('SCHEDULED EVENT (RECURRING)', '-', inplace=True)
    
# Frisk Flag
df['Frisk Flag'].replace('-', np.nan, inplace=True)

# Precinct
df['Precinct'].replace('-', 'Unknown', inplace=True)
df['Precinct'].replace('FK ERROR', 'Unknown', inplace=True)
#df['Precinct'].replace('OOJ', 'Unknown', inplace=True)
df['Precinct'].replace('SouthWest', 'Southwest', inplace=True)

# Sector
df['Sector'].replace('99', '-', inplace=True)

# Beat
df['Beat'].replace('S', '-', inplace=True)
df['Beat'].replace('99', '-', inplace=True)


# Categorize "Call Types". Variable used to create dictionary in CSV
call_types = pd.concat([df['Call Type'], df['Initial Call Type'], df['Final Call Type']]).value_counts().keys().values
path = os.path.join('..', 'data', 'categorize call types.csv')
call_type_dictionary = pd.read_csv(path)
call_type_dictionary = dict(zip(call_type_dictionary.iloc[:, 0], call_type_dictionary.iloc[:, 1]))


    
#--- INCOMPLETE!
df['Initial Call Type'] = df['Initial Call Type'].map(call_type_dictionary)
df['Final Call Type'] = df['Final Call Type'].map(call_type_dictionary)
df['Call Type'] = df['Call Type'].map(call_type_dictionary)


In [49]:
# Engineer new features

# Calculate Officer Age
if 'Officer YOB' in df.columns:
    df['Officer Age'] = df['Reported Date'].apply(lambda x: x.year) - df['Officer YOB']
    df.drop('Officer YOB', axis=1, inplace=True)

    
# Determine whether or not weapon was involved
has_weapon = lambda x: 'No' if x == 'None' else 'Yes'
df['Weapon (Y/N)'] = df['Weapon Type'].apply(has_weapon)


# Calculate the number of stops per subject
offense_ct_dict = dict(df['Subject ID'].value_counts())
df['Offense Count'] = df['Subject ID'].map(offense_ct_dict)


# Identify whether a subject has been involved in multiple stops
repeat_offenders = lambda offense_ct: 'Yes' if offense_ct > 1 else 'No'
df['Repeat Offender (Y/N)'] = df['Offense Count'].apply(repeat_offenders)

In [52]:
# Refine dataset to continous and OHE values

# Separate columns by type
# Identify continuous vs. string type features
str_columns = []
continuous_columns = []
for col in df.columns:
    if type(df.dropna()[col].iloc[0]) == type('abc'):
        str_columns.append(col)
    else:
        continuous_columns.append(col)
# Determine which columns are most valuable to OHE
cols_to_ohe = []
for col in str_columns:
    if len(df[col].value_counts().index) < 20:
        cols_to_ohe.append(col)
        
df_refined = pd.concat([df[continuous_columns], df[cols_to_ohe]], axis=1)

In [57]:
# Drop "Offense Count" for now given the number of missing values
if 'Offense Count' in df_refined:
    df_refined.drop('Offense Count', axis=1, inplace=True)
    
# Drop all missing values from dataframe
df_refined.dropna(inplace=True)

In [58]:
df_refined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45134 entries, 214 to 47212
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Reported Date             45134 non-null  datetime64[ns]
 1   Reported Time             45134 non-null  object        
 2   Officer Age               45134 non-null  int64         
 3   Subject Age Group         45134 non-null  object        
 4   Stop Resolution           45134 non-null  object        
 5   Weapon Type               45134 non-null  object        
 6   Officer Gender            45134 non-null  object        
 7   Officer Race              45134 non-null  object        
 8   Subject Perceived Race    45134 non-null  object        
 9   Subject Perceived Gender  45134 non-null  object        
 10  Call Type                 45134 non-null  object        
 11  Arrest Flag               45134 non-null  object        
 12  Frisk Flag      